# Domino Shuffle algorithm to generate AztecDiamonds

The cell below details the shuffling algorithm for generating Azetc Diamonds of any rank N

In [2]:
"""
The Aztec Diamond is represented as a dictionary
where the key is the coordinate location of the 
bottom left point of each tile and the value is the
orientation/sliding direction of the tile u:up,d:down,l:left,r:right,None:empty tile
The coordinate plane I use to map the tiles has the bottom left point of the middle bloc centered
at 0,0

Lets take the rank 1 tiling for example. This tiling has 2 possible reprsentations
The dictionary representation therefore can either be
{-1,-1:d, 0,-1:d, -1,0:u, 0,0:u} or
{-1,-1:l, 0,-1:r, -1,0:l, 0,0:r}

"""

import random

def domino_shuffle(n):
    tiling = {}
    tiles = []
    
    #iterate through the ranks and generate boards until we get rank desired
    for rank in range(1,n+1):
        tiling,tiles = create_board(rank,tiling, tiles)
    return tiling

"""
Each time I call create_board, I create a new coordinate system for the new rank
I then create an empty dictionary and populate its keys with the coordinates I just generated
I then call destroy,slide,fill to generate the orientations for the new board
"""
def create_board(rank,prev_tiling,prev_tiles):
    tiles = []
    #generate coordinate system
    for i in range(-rank, rank):
        m = min(rank + 1 + i, rank - i)
        for j in range(-m, m):
            tiles.append((j, i))

    tiling = {tile: None for tile in tiles}
    
    if(rank != 1):
        prev_tiling = destroy(prev_tiles,prev_tiling)
        tiling = slide(prev_tiles,tiling,prev_tiling)     
        
    return fill(tiles,tiling),tiles

"""
Based on the tiles orientation from the previous board, I slide the tiles and update
accordingly in the new board
"""       
def slide(prev_tiles,tiling,prev_tiling):
    for (i,j) in prev_tiles:
        if prev_tiling[(i, j)] == "u":
            tiling[(i, j + 1)] = "u"
        if prev_tiling[(i, j)] == "d":
            tiling[(i, j - 1)] = "d"
        if prev_tiling[(i, j)] == "l":
            tiling[(i - 1, j)] = "l"
        if prev_tiling[(i, j)] == "r":
            tiling[(i + 1, j)] = "r"
    
    return tiling

"""
I check if any tiles in the previous tiling are oriented towards each other, in other words
they will slide into each other during the slide step. If so I delete all these tiles by setting
their values to None
"""
def destroy(tiles, tiling):
    for i, j in tiles:
        try:
            if check_block(i, j, tiling,["u", "u", "d", "d"]) or check_block(
                    i, j, tiling,["r", "l", "r", "l"]):
                    fill_helper(i, j, tiling,[None] * 4)
        except KeyError:
                pass
    return tiling

"""
Helper method to check if block or 4 tiles have a certain set of values
"""
def check_block(i,j,tiling,domino_values):
    if (tiling[(i,j)] == domino_values[0] and tiling[(i+1,j)] == domino_values[1] and
        tiling[(i,j+1)] == domino_values[2] and tiling[(i+1,j+1)] == domino_values[3]):
        return True
    return False

"""
I fill the remaining empty tiles by checking for groups of empty 4 tiles or blocks
If I find an empty block I randomly pick a rank 1 tiling to fill into the block
"""
def fill(tiles,tiling):
    for i, j in tiles:
            try:
                rint = random.randrange(2)
                if check_block(i, j, tiling, [None] * 4):
                    if rint == 0:
                        tiling = fill_helper(i, j, tiling, ["d", "d", "u", "u"])
                    else:
                        tiling = fill_helper(i, j, tiling, ["l", "r", "l", "r"])
            except KeyError:
                pass
    return tiling
            
"""
Helper method for fill
"""
def fill_helper(i,j,tiling,domino_values):
    tiling[(i,j)] = domino_values[0]
    tiling[(i+1,j)] = domino_values[1]
    tiling[(i,j+1)] = domino_values[2]
    tiling[(i+1,j+1)] = domino_values[3]
    
    return tiling
    

Here we test out the algorithm by creating a tiling of rank 100. The algorithm prints out the board starting from the bottom left most point and moving from top to bottom

In [4]:
domino_shuffle(3)

{(-1, -3): 'd',
 (0, -3): 'd',
 (-2, -2): 'd',
 (-1, -2): 'd',
 (0, -2): 'l',
 (1, -2): 'r',
 (-3, -1): 'l',
 (-2, -1): 'u',
 (-1, -1): 'u',
 (0, -1): 'l',
 (1, -1): 'r',
 (2, -1): 'r',
 (-3, 0): 'l',
 (-2, 0): 'l',
 (-1, 0): 'u',
 (0, 0): 'u',
 (1, 0): 'r',
 (2, 0): 'r',
 (-2, 1): 'l',
 (-1, 1): 'l',
 (0, 1): 'r',
 (1, 1): 'r',
 (-1, 2): 'l',
 (0, 2): 'r'}

In [ ]:
domino_shuffle(100)

# Height function

There is a bijection between Aztec Diamond tilings and the height function, which assigns a numerical value to each of the vertices of the domino based on some set rules. We can see that the height along the boundaries does not depend on the tiling and is linear.

In [67]:
"""
Height function #2
"""

def get_heights(tiling, rank):
    
    height_tiles = []
    M = 2 * rank + 1
    L = rank
    #generate height board
    for i in range(-rank, rank+1):
        m = min(rank + 1 + i, rank+1 - i)
        if i == 0:
            m = rank
        for j in range(-m, m+1)
        height_tiles.append((j,i))
        
    height_board = {tile: None for tile in height_tiles}
    
    height_board[(0,-rank)] = 0
    
    # fill in boundary
    for k in range(0,L):
        # top-left
        height_board[k]
        heights[(L-k)*M + k] = 2*(L-k)
        heights[(L-k-1)*M + k] = 2*(L-k)-1
        # top-right
        heights[k*M + L+k] = 2*k
        heights[k*M + L+k+1] = 2*k+1
        # bottom-right
        heights[(L+k)*M + 2*L-k] = 2*(L-k)
        heights[(L+k+1)*M + 2*L-k] = 2*(L-k)-1
        # bottom-left
        heights[(2*L-k)*M + L-k] = 2*k
        heights[(2*L-k)*M + L-k-1] = 2*k+1

In [5]:
"""
Height function
"""

def compute_height(tiling, rank):
    
    height_tiles = []
    # generate height board
    for i in range(-rank, rank+1):
        m = min(rank + 1 + i, rank+1 - i)
        if i == 0:
            m = rank
        for j in range(-m, m+1):
            height_tiles.append((j, i))

    height_board = {tile: None for tile in height_tiles}
    
    height_board[(0,-rank)] = 0
    
    skip_flag = 0;
    for j in range(-rank, rank):
        m = min(rank + 1 + j, rank - j)
        for i in range(-m, m):
            if(skip_flag == 1):
                skip_flag = 0
                continue
            ref_point = i
            if i < 0:
                ref_point = i+1
            
            try:
                if(tiling[(i,j)] == "d" and tiling[(i+1,j)] == "d"):
                    height_board = set_height(i,j,height_board,ref_point,"d")
                    skip_flag = 1
                elif(tiling[(i,j)] == "u" and tiling[(i+1,j)] == "u"):
                    height_board = set_height(i,j,height_board,ref_point,"u")
                    skip_flag = 1
                elif(tiling[(i,j)] == "l" and tiling[(i,j+1)] == "l"):
                    try:
                        if(tiling[(i,j-1)] != "l"):
                            height_board = set_height(i,j,height_board,ref_point,"l")
                    except KeyError:
                        height_board = set_height(i,j,height_board,ref_point,"l")
                        pass
                elif(tiling[(i,j)] == "r" and tiling[(i,j+1)] == "r"):
                    try:
                        if(tiling[(i,j-1)] != "r"):
                            height_board = set_height(i,j,height_board,ref_point,"r")
                    except KeyError:
                        height_board = set_height(i,j,height_board,ref_point,"r")
                        pass
                      
            except KeyError:
                pass
            
    return height_board


def set_height(i,j,height_board,ref_point, pos):
    if pos == "d":
        h = height_board[(ref_point,j)]
        if ref_point == i+1:
            height_board[(i,j)] = h + 1
            height_board[(i+2,j)] = h + 1
            height_board[(i+2,j+1)] = h + 2
            height_board[(i+1,j+1)] = h + 3
            height_board[(i,j+1)] = h + 2
        else:
            height_board[(i+1,j)] = h - 1
            height_board[(i+2,j)] = h 
            height_board[(i+2,j+1)] = h + 1
            height_board[(i+1,j+1)] = h + 2
            height_board[(i,j+1)] = h + 1
    elif pos == "u":
        h = height_board[(ref_point,j)]
        if ref_point == i+1:
            height_board[(i,j)] = h - 1
            height_board[(i+2,j)] = h - 1
            height_board[(i+2,j+1)] = h - 2
            height_board[(i+1,j+1)] = h - 3
            height_board[(i,j+1)] = h - 2
        else:
            height_board[(i+1,j)] = h + 1
            height_board[(i+2,j)] = h 
            height_board[(i+2,j+1)] = h - 1
            height_board[(i+1,j+1)] = h - 2
            height_board[(i,j+1)] = h - 1
    
    elif pos == "l":
        h = height_board[(ref_point,j)]
        if ref_point == i+1:
            height_board[(i,j)] = h + 1
            height_board[(i,j+1)] = h + 2
            height_board[(i,j+2)] = h + 1
            height_board[(i+1,j+2)] = h
            height_board[(i+1,j+1)] = h - 1
        else:
            height_board[(i,j+1)] = h + 1
            height_board[(i,j+2)] = h
            height_board[(i+1,j+2)] = h - 1
            height_board[(i+1,j+1)] = h - 2
            height_board[(i+1,j)] = h - 1
    
    elif pos == "r":
        h = height_board[(ref_point,j)]
        if ref_point == i+1:
            height_board[(i,j)] = h - 1
            height_board[(i,j+1)] = h - 2
            height_board[(i,j+2)] = h - 1
            height_board[(i+1,j+2)] = h
            height_board[(i+1,j+1)] = h + 1
        else:
            height_board[(i,j+1)] = h - 1
            height_board[(i,j+2)] = h
            height_board[(i+1,j+2)] = h + 1
            height_board[(i+1,j+1)] = h + 2
            height_board[(i+1,j)] = h + 1

    return height_board


Creating a rank 3 AD and generating heights for each vertex

In [6]:
rank = 3
tiling = domino_shuffle(rank)
height_board = compute_height(tiling,rank)
print(tiling)
print()
print(height_board)

{(-1, -3): 'd', (0, -3): 'd', (-2, -2): 'd', (-1, -2): 'd', (0, -2): 'd', (1, -2): 'd', (-3, -1): 'l', (-2, -1): 'r', (-1, -1): 'l', (0, -1): 'r', (1, -1): 'd', (2, -1): 'd', (-3, 0): 'l', (-2, 0): 'r', (-1, 0): 'l', (0, 0): 'r', (1, 0): 'u', (2, 0): 'u', (-2, 1): 'u', (-1, 1): 'u', (0, 1): 'u', (1, 1): 'u', (-1, 2): 'u', (0, 2): 'u'}

{(-1, -3): 1, (0, -3): 0, (1, -3): 1, (-2, -2): 3, (-1, -2): 2, (0, -2): 3, (1, -2): 2, (2, -2): 3, (-3, -1): 5, (-2, -1): 4, (-1, -1): 5, (0, -1): 4, (1, -1): 5, (2, -1): 4, (3, -1): 5, (-3, 0): 6, (-2, 0): 3, (-1, 0): 6, (0, 0): 3, (1, 0): 6, (2, 0): 7, (3, 0): 6, (-3, 1): 5, (-2, 1): 4, (-1, 1): 5, (0, 1): 4, (1, 1): 5, (2, 1): 4, (3, 1): 5, (-2, 2): 3, (-1, 2): 2, (0, 2): 3, (1, 2): 2, (2, 2): 3, (-1, 3): 1, (0, 3): 0, (1, 3): 1}
